In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [ ]:
df = pd.read_csv(r"C:\Users\pushpak\Downloads\11000 Medicine details COPY_exported\11000 Medicine details COPY_exported.csv")
print(len(df)) # 192368

embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
df = pd.read_csv(r"C:\Users\pushpak\Downloads\11000 Medicine details COPY_exported\11000 Medicine details COPY_exported.csv")
print(len(df)) # 192

In [ ]:
corpus = df["Medicine Name"] + " " + df["Composition"] + " " + df["Uses"]

In [ ]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [ ]:
print(corpus_embeddings.shape)

In [ ]:
import torch
torch.save(corpus_embeddings,r"C:\Users\pushpak\Documents\emails_data\ui\embeddings.pth")

In [ ]:
query = "small dick"
top_k = 10

# Find the closest top_k sentences of the corpus based on cosine similarity
query_embedding = embedder.encode(query, convert_to_tensor=True)
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
hits = hits[0]

print(f"\nTop {top_k} most similar sentences in corpus:")
for hit in hits:
    hit_id = hit['corpus_id']
    article_data = df.iloc[hit_id]
    title = article_data["Medicine Name"]
    print("-", title, "(Score: {:.4f})".format(hit['score']))

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key="  ")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a doctor. you will look into the problem provided by the user and summarize the problem in just two medical terms"),
    ("user", "{input}")
])

chain = prompt | llm 

In [ ]:
%%time
query = chain.invoke({"input": "hair loss with dandruff"}).content
print(query)
top_k = 10

# Find the closest top_k sentences of the corpus based on cosine similarity
query_embedding = embedder.encode(query, convert_to_tensor=True)
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
hits = hits[0]

print(f"\nTop {top_k} most similar sentences in corpus:")
for hit in hits:
    hit_id = hit['corpus_id']
    data = df.iloc[hit_id]
    title = data["Medicine Name"]
    print("-", title, "(Score: {:.4f})".format(hit['score']))